# im2latex(S): Tokenizer

&copy; Copyright 2017 Sumeet S Singh

    This file is part of im2latex solution by Sumeet S Singh.

    This program is free software: you can redistribute it and/or modify
    it under the terms of the Affero GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    Affero GNU General Public License for more details.

    You should have received a copy of the Affero GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string

In [2]:
data_dir = '../data/generated2'

#### ASCII Control Characters

                        CTRL   (^D means to hold the CTRL key and hit d)
    Oct  Dec Char  Hex  Key     Comments
    \000   0  NUL  \x00  ^@ \0 (Null byte)
    \001   1  SOH  \x01  ^A    (Start of heading)
    \002   2  STX  \x02  ^B    (Start of text)
    \003   3  ETX  \x03  ^C    (End of text) (see: UNIX keyboard CTRL)
    \004   4  EOT  \x04  ^D    (End of transmission) (see: UNIX keyboard CTRL)
    \005   5  ENQ  \x05  ^E    (Enquiry)
    \006   6  ACK  \x06  ^F    (Acknowledge)
    \007   7  BEL  \x07  ^G    (Ring terminal bell)
    \010   8   BS  \x08  ^H \b (Backspace)  (\b matches backspace inside [] only)
                                            (see: UNIX keyboard CTRL)
    \011   9   HT  \x09  ^I \t (Horizontal tab)
    \012  10   LF  \x0A  ^J \n (Line feed)  (Default UNIX NL) (see End of Line below)
    \013  11   VT  \x0B  ^K    (Vertical tab)
    \014  12   FF  \x0C  ^L \f (Form feed)
    \015  13   CR  \x0D  ^M \r (Carriage return)  (see: End of Line below)
    \016  14   SO  \x0E  ^N    (Shift out)
    \017  15   SI  \x0F  ^O    (Shift in)
    \020  16  DLE  \x10  ^P    (Data link escape)
    \021  17  DC1  \x11  ^Q    (Device control 1) (XON) (Default UNIX START char.)
    \022  18  DC2  \x12  ^R    (Device control 2)
    \023  19  DC3  \x13  ^S    (Device control 3) (XOFF)  (Default UNIX STOP char.)
    \024  20  DC4  \x14  ^T    (Device control 4)
    \025  21  NAK  \x15  ^U    (Negative acknowledge)  (see: UNIX keyboard CTRL)
    \026  22  SYN  \x16  ^V    (Synchronous idle)
    \027  23  ETB  \x17  ^W    (End of transmission block)
    \030  24  CAN  \x18  ^X    (Cancel)
    \031  25  EM   \x19  ^Y    (End of medium)
    \032  26  SUB  \x1A  ^Z    (Substitute character)
    \033  27  ESC  \x1B  ^[    (Escape)
    \034  28  FS   \x1C  ^\    (File separator, Information separator four)
    \035  29  GS   \x1D  ^]    (Group separator, Information separator three)
    \036  30  RS   \x1E  ^^    (Record separator, Information separator two)
    \037  31  US   \x1F  ^_    (Unit separator, Information separator one)
    \177 127  DEL  \x7F  ^?    (Delete)  (see: UNIX keyboard CTRL)

In [3]:
for i in range(32):
    if chr(i) in string.printable:
        print i

9
10
11
12
13


In [4]:
for i in range(32):
    if chr(i) in string.whitespace:
        print i

9
10
11
12
13


In [5]:
LATEX_RE = re.compile(r"(?:\\[a-zA-Z]+)|.")
DELETE_RE = re.compile(r".\x7F")

In [6]:
def get_df_clean(data_dir_):
    try:
        return pd.read_pickle(os.path.join(data_dir_,'image_details_clean.pkl'))
    except:
        pass
    
    def cleanAscii(s):
        # Throw exception if any non-ascii characters exist
        s = codecs.encode(codecs.decode(s, 'ascii'), 'ascii') # throws
        # Process the deletes in the latex formulas - there are a few
        s = re.sub(DELETE_RE, '', s)
        # Remove non-printable characters (e.g. \r was found in the data)
        ls = [ ''.join(c for c in ss if c in string.printable) for ss in s.split()]
        # Normalize continuous whitespaces (i.e. multiple \t or spaces) with a single space
        s = ' '.join(ls)
        return pd.Series([s, len(s)], index=['latex_ascii', 'latex_ascii_len'])

    def clean(df):
        # Throw exception if any non-ascii characters exist
        # Process deletes - there are a few
        # Replace continuous whitespace with a single space
        cleaned = df.latex.str.decode('ascii').str.encode('ascii').str.replace(DELETE_RE, '').str.replace(r"\s+", ' ').str.strip()
        return df.assign(latex_ascii=cleaned, latex_ascii_len=cleaned.str.len())
        
        
    #cleaned = df.latex.apply(cleanAscii)
    #df_clean = df.assign(latex_ascii=cleaned.latex_ascii, latex_ascii_len=cleaned.latex_ascii_len)
    #df_clean.to_pickle(os.path.join(data_dir_,'image_details_clean.pkl'))
    #return df_clean
    return clean(pd.read_pickle(os.path.join(data_dir_,'image_details.pkl')))

In [7]:
def get_df_clean2(data_dir_):
#     try:
#         return pd.read_pickle(os.path.join(data_dir_,'image_details_clean.pkl'))
#     except:
#         pass
    
    df = pd.read_pickle(os.path.join(data_dir_,'image_details.pkl'))

    def cleanAscii(s):
        # Throw error if any non-ascii characters exits
        s = codecs.encode(codecs.decode(s, 'ascii'), 'ascii') # throws
        # Remove ascii control characters (e.g. \r was found in the data)
        s = ' '.join(s.split())
        return pd.Series([s, len(s)], index=['latex_ascii', 'latex_ascii_len'])
    
    cleaned = df.latex.apply(cleanAscii)
    df_clean = df.assign(latex_ascii=cleaned.latex_ascii, latex_ascii_len=cleaned.latex_ascii_len)
    #df_clean.to_pickle(os.path.join(data_dir_,'image_details_clean.pkl'))
    return df_clean

In [8]:
df_clean = get_df_clean(data_dir)
display(df_clean.shape)

(102180, 8)

In [ ]:
df_clean2 = get_df_clean2(data_dir)
display(df_clean2.shape)

In [ ]:
df_diff = df_clean.latex != df_clean2.latex

In [ ]:
sum(df_diff)

In [ ]:
def get_df_dirty(data_dir_, df_clean_):
    try:
        return pd.read_pickle(os.path.join(data_dir_,'image_details_dirty.pkl'))
    except Exception as e:
        print('Exception:',e)
    
    df_dirty = df_clean_[(df_clean_.latex != df_clean_.latex_ascii)]
    df_dirty = df_dirty[['latex', 'latex_ascii', 'formula_len', 'latex_ascii_len']]
    df_dirty.to_pickle(os.path.join(data_dir_,'image_details_dirty.pkl'))
    return df_dirty

In [ ]:
def get_df_dirty2(data_dir_, df_clean_):
#     try:
#         return pd.read_pickle(os.path.join(data_dir_,'image_details_dirty.pkl'))
#     except Exception as e:
#         print('Exception:',e)
    
    df_dirty = df_clean_[(df_clean_.latex != df_clean_.latex_ascii)]
    df_dirty = df_dirty[['latex', 'latex_ascii', 'formula_len', 'latex_ascii_len']]
    #df_dirty.to_pickle(os.path.join(data_dir_,'image_details_dirty.pkl'))
    return df_dirty

In [ ]:
df_dirty = get_df_dirty2(data_dir, df_clean)
display(df_dirty.shape)
display(df_dirty.head())

In [9]:
def extractTokensFromStr(s):
    return set(re.findall(LATEX_RE, s))

def get_vocabulary(data_dir_, df_clean_):
    try:
        with open(os.path.join(data_dir_, 'vocabulary.pkl'), 'rb') as f:
            return pickle.load(os.path.join(data_dir_, 'vocabulary.pkl'))
    except Exception as e:
        print('Exception:', e)
        
    tokenS = df_clean_.latex_ascii.apply(extractTokensFromStr)
    words = set([])
    tokenS.agg(lambda s: words.update(s))
#     with open(os.path.join(data_dir_, 'vocabulary.pkl'), 'wb') as f:
#         pickle.dump(words, f, pickle.HIGHEST_PROTOCOL)
    return words
#extractTokensFromStr('xyz\\abc  \tefg{}\r')

In [ ]:
def extractTokensFromStr2(s):
    return set(re.findall(LATEX_RE, ' '.join(s.split())))

def get_vocabulary2(data_dir_, df_clean_):
#     try:
#         with open(os.path.join(data_dir_, 'vocabulary.pkl'), 'rb') as f:
#             return pickle.load(os.path.join(data_dir_, 'vocabulary.pkl'))
#     except Exception as e:
#         print('Exception:', e)
        
    tokenS = df_clean_.latex_ascii.apply(extractTokensFromStr2)
    words = set([])
    tokenS.agg(lambda s: words.update(s))
#     with open(os.path.join(data_dir_, 'vocabulary.pkl'), 'wb') as f:
#         pickle.dump(words, f, pickle.HIGHEST_PROTOCOL)
    return words

In [ ]:
vocabulary = get_vocabulary(data_dir, df_clean)

In [ ]:
len(vocabulary)

In [ ]:
vocabulary2 = get_vocabulary2(data_dir, df_clean2)

In [ ]:
len(vocabulary2)

In [ ]:
for word in vocabulary2:
    if word not in vocabulary:
        print ord(word)

In [ ]:
for word in vocabulary:
    if word not in vocabulary2:
        print ord(word)

In [16]:
s_dirty = df_clean.latex_ascii[df_clean.latex.str.contains(chr(127))]
for i in range(s_dirty.shape[0]):
    display(s_dirty.iloc[i])

'\\label{vvvfq}\\xi_q:=\\delta_v q^i(t) \\frac{\\partial}{\\partial q^i},%'

'\\label{lied1}\\delta_h F(q^i(t), \\dot q^i(t), t)={L}_\\xi F(q^i(t), \\dot q^i(t), t).%'

'\\label{dSD}d_v S_D=\\sum_k (t_{k+1}-t_k)d_v {L_D}^{(k)}, \\qquad d_v{L_D}^{(k)}={{\\cal E}_D}^{(k)}+\\Delta_k{\\theta_D}^{(k)},'

'\\label{ddsDf}d_v{{\\cal E}_D}^{(i,j)}+\\Delta_\\mu {\\omega_D}^{\\mu (i,j)}=0,\\qquad{\\omega_D}^{\\mu(i,j)}:=d_v{\\theta_D}^{\\mu(i,j)}.%=d_v{p_{i}}^{(k+1)}\\wedge d_vq^{i(k)}.% .'

'\\label{eleqm}\\frac{\\partial {L}}{\\partial q^i}-\\frac{d}{dt}(\\frac{\\partial{L}}{\\partial \\dot q^i})=0.%'

'\\label{dDt}\\delta_t(t_{k+1}-t_k)=\\Delta_k(\\delta_tt_k)(t_{k+1}-t_k),%'